In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('echo_ps_matches_all_notes07Feb18.csv')
texts = df.groupby('ICUSTAY_ID')['TEXT'].apply(lambda x: x.sum()).to_frame()

In [8]:
# df_match = pd.read_csv('PSMatchDetails.csv')
# len(set(df_match['icustay_id']))

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(analyzer='char_wb', ngram_range=(2,2), min_df = 0)

In [11]:
bigrams = cv.fit_transform(texts['TEXT'])

In [12]:
bigrams.toarray().shape

(4363, 4341)

In [61]:
cv = CountVectorizer(analyzer='char_wb', ngram_range=(3,3), min_df = 0)

In [62]:
trigrams = cv.fit_transform(df_master['TEXT_all'])

In [63]:
df_master.to_csv('master_texts.csv')

In [13]:
import numpy as np
np.savetxt('bigrams.csv',bigrams.toarray(),delimiter=',')
# np.savetxt('trigrams.csv',trigrams.toarray(),delimiter=',')